In [1]:
# Retraining NN with Elegy then do HMCon Mana
# Author: Peter Nov 5 2022
# New Requirements:
# conda install python=3.9 numpy scipy pandas matplotlib
# pip install elegy==0.8.5 # Because 0.8.6 has error.
# pip install --upgrade "jax[cuda]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
# pip install tfp-nightly tensorflow_io tensorflow

from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
import os
#os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'
#os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = '.10'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import jax
import jax.numpy as jnp
from jax import random
from jax import vmap
from jax import jit
from jax import grad
#assert jax.default_backend() == 'gpu'

import elegy # pip install elegy
import optax
import tensorflow_io as tfio # pip install tensorflow-io
#import tensorflow as tf # Recommended not to import this with jax because will also try to grab memory.
from tensorflow.data import Dataset # Trying not to import tf. 
from tensorflow.data.experimental import AUTOTUNE

2022-11-06 00:40:15.491547: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-06 00:40:16.352175: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/psadow/libs/anaconda3/envs/herox/lib
2022-11-06 00:40:16.352307: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/psadow/libs/anaconda3/envs/herox/lib
2022-11-06 00:40:16.352316: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the

# Retrain model with Elegy 0.8.5 version. (0.8.6 has bugs)

In [2]:
# Define dataset
f = '/mnt/lts/nfs_fs02/sadow_lab/shared/gcr/data/proposal/data_processed.hdf5'
x = tfio.IODataset.from_hdf5(f, dataset='/X_minmax')
y = tfio.IODataset.from_hdf5(f, dataset='/Y_logp1_scaled')
full = Dataset.zip((x, y))
# Split
train = full.take(np.floor(1435308 *.9)).repeat()
test = full.skip(np.floor(1435308 *.9)).repeat()

# Batch
BATCH_SIZE = 128
train = train.batch(BATCH_SIZE, drop_remainder=True).prefetch(AUTOTUNE)
test = test.batch(BATCH_SIZE, drop_remainder=True).prefetch(AUTOTUNE)

#train_x = TFDatasetAdapter(train.map(lambda x,y: x))
#train_y = TFDatasetAdapter(train.map(lambda x,y: x))
#x = train.map(lambda x,y: x)
#y = train.map(lambda x,y: y)

class MLP(elegy.Module):
    def __init__(self):
        pass
    
    @elegy.compact
    def __call__(self, x: jnp.ndarray) -> jnp.ndarray:
        #initializer = elegy.initializers.RandomNormal(stddev=.001)
        #initializer = None #elegy.initializers.TruncatedNormal()
        x = x.astype(jnp.float32) 
        x = elegy.nn.Flatten()(x)
        
        x = elegy.nn.Linear(256)(x) 
        x = jax.nn.selu(x)
        #x = elegy.nn.Dropout(0.5)(x)
        
        x = elegy.nn.Linear(256)(x)
        x = jax.nn.selu(x)
        #x = elegy.nn.Dropout(0.3)(x)
        
        x = elegy.nn.Linear(245)(x)
        return x

model = elegy.Model(
    # Creates stateful model and optimizer. See elegy docs.
    module=MLP(),
    loss=[elegy.losses.MeanSquaredError(),
          #elegy.losses.MeanAbsoluteError(),
          elegy.regularizers.L2(l=1e-6),
         ],
    #loss=[elegy.losses.MeanSquaredError(), elegy.regularizers.GlobalL2(l=1e-5)]
    #metrics=elegy.metrics.BinaryAccuracy(),
    optimizer=optax.adam(1e-4), 
    #optimizer=optax.sgd(1e-10),
    #optimizer=optimizers.rmsprop(1e-3), # Oryx based on optax. Doesn't work with elegy.
    #run_eagerly=True, # Seems to be necessary for 
)
path = 'model_2_256_selu_l21e-6'
callbacks = [elegy.callbacks.EarlyStopping(monitor="val_mean_squared_error_loss", patience=5),
             elegy.callbacks.ModelCheckpoint(path=path, save_best_only=True, monitor='val_mean_squared_error_loss'),
             #elegy.callbacks.TensorBoard("summaries"),
            ]

history = model.fit(
    inputs=train,
    epochs=50,
    steps_per_epoch= 10000, #6 * 10000, #10000, # 10k*128 is approximate size of training set.
    validation_data=test,
    validation_steps=1000,
    shuffle=False,
    verbose=2,
    callbacks=callbacks,
)

# Save model. 
#model.save('model')  # creates folder at 'my_model'
#del model  # deletes the existing model
# Test ability to make prediction.
# x = np.random.rand((7))  # This used to work but no longer.
x = np.random.rand((1,7))
model.predict(x).shape

2022-11-06 00:01:01.002977: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available
2022-11-06 00:01:02.599150: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.


Epoch 1/50


/home/psadow/libs/anaconda3/envs/jax_2023/lib/python3.9/site-packages/elegy/data/utils.py:199: FutureWarning: jax.tree_leaves is deprecated, and will be removed in a future release. Use jax.tree_util.tree_leaves instead.
  sizes: tp.Set[int] = set(x.shape[0] for x in jax.tree_leaves(batch_elems))
/home/psadow/libs/anaconda3/envs/jax_2023/lib/python3.9/site-packages/flax/core/scope.py:740: FutureWarning: jax.tree_leaves is deprecated, and will be removed in a future release. Use jax.tree_util.tree_leaves instead.
  abs_value_flat = jax.tree_leaves(abs_value)
/home/psadow/libs/anaconda3/envs/jax_2023/lib/python3.9/site-packages/flax/core/scope.py:741: FutureWarning: jax.tree_leaves is deprecated, and will be removed in a future release. Use jax.tree_util.tree_leaves instead.
  value_flat = jax.tree_leaves(value)
/home/psadow/libs/anaconda3/envs/jax_2023/lib/python3.9/site-packages/treex/optimizer.py:79: FutureWarning: jax.tree_leaves is deprecated, and will be removed in a future release

10000/10000 - 26s - l2_loss: 7.2923e-04 - loss: 0.0139 - mean_squared_error_loss: 0.0132 - val_l2_loss: 5.6836e-04 - val_loss: 6.1758e-04 - val_mean_squared_error_loss: 4.9224e-05
Epoch 2/50
10000/10000 - 18s - l2_loss: 4.2084e-04 - loss: 4.4084e-04 - mean_squared_error_loss: 2.0001e-05 - val_l2_loss: 2.8861e-04 - val_loss: 2.9669e-04 - val_mean_squared_error_loss: 8.0803e-06
Epoch 3/50
10000/10000 - 18s - l2_loss: 1.9766e-04 - loss: 2.0415e-04 - mean_squared_error_loss: 6.4848e-06 - val_l2_loss: 1.2741e-04 - val_loss: 1.3228e-04 - val_mean_squared_error_loss: 4.8690e-06
Epoch 4/50
10000/10000 - 18s - l2_loss: 8.8061e-05 - loss: 9.2759e-05 - mean_squared_error_loss: 4.6980e-06 - val_l2_loss: 5.9105e-05 - val_loss: 6.3637e-05 - val_mean_squared_error_loss: 4.5319e-06
Epoch 5/50
10000/10000 - 18s - l2_loss: 4.4152e-05 - loss: 4.8342e-05 - mean_squared_error_loss: 4.1901e-06 - val_l2_loss: 3.3598e-05 - val_loss: 3.7435e-05 - val_mean_squared_error_loss: 3.8367e-06
Epoch 6/50
10000/10000 -

ScopeParamShapeError: Inconsistent shapes between value and initializer for parameter "kernel" in "/": (7, 256), (1, 256). (https://flax.readthedocs.io/en/latest/flax.errors.html#flax.errors.ScopeParamShapeError)

In [6]:
model_path = './model_2_256_selu_l21e-6/'
model = elegy.load(model_path)
x = np.random.rand(1,7)
model.predict(x).shape

/home/psadow/libs/anaconda3/envs/jax_2023/lib/python3.9/site-packages/flax/core/scope.py:740: FutureWarning: jax.tree_leaves is deprecated, and will be removed in a future release. Use jax.tree_util.tree_leaves instead.
  abs_value_flat = jax.tree_leaves(abs_value)
/home/psadow/libs/anaconda3/envs/jax_2023/lib/python3.9/site-packages/flax/core/scope.py:741: FutureWarning: jax.tree_leaves is deprecated, and will be removed in a future release. Use jax.tree_util.tree_leaves instead.
  value_flat = jax.tree_leaves(value)


(1, 245)